In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Luderitz,-26.65,15.16,55.40,76,52,9.17,NaN,2019-11-08 04:49:33
1,1,Rikitea,-23.12,-134.97,74.55,82,81,9.91,PF,2019-11-08 04:49:33
2,2,Bluff,-23.58,149.07,98.49,7,0,15.32,AU,2019-11-08 04:45:49
3,3,Jumla,29.28,82.18,58.77,30,32,6.35,NP,2019-11-08 04:49:33
4,4,Puerto Quijarro,-19.00,-57.72,72.66,97,83,2.77,BO,2019-11-08 04:49:33


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp]) #[max(0,temp) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds=city_data_df["Cloudiness"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, 
weights=clouds,
dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.

fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,Tiarei,-17.53,-149.33,84.20,62,75,8.05,PF,2019-11-08 04:49:34
8,8,Avarua,-21.21,-159.78,75.20,78,100,9.17,CK,2019-11-08 04:49:34
9,9,Hervey Bay,-25.30,152.85,86.00,48,0,25.28,AU,2019-11-08 04:49:34
11,11,Georgetown,6.80,-58.16,75.20,100,20,2.24,GY,2019-11-08 04:47:55
12,12,Kapaa,22.08,-159.32,78.80,65,75,8.05,US,2019-11-08 04:49:35
16,16,Vaini,15.34,74.49,75.20,78,40,0.76,IN,2019-11-08 04:49:36
32,32,Itupiranga,-5.13,-49.33,78.80,94,63,1.34,BR,2019-11-08 04:49:40
34,34,Bongandanga,1.51,21.07,75.29,84,71,2.21,CD,2019-11-08 04:49:40
41,41,Naze,5.43,7.07,75.45,94,78,1.81,NG,2019-11-08 04:49:42
42,42,Atuona,-9.80,-139.03,80.58,78,0,14.56,PF,2019-11-08 04:49:42


In [11]:
preferred_cities_df.count()

City_ID       146
City          146
Lat           146
Lng           146
Max Temp      146
Humidity      146
Cloudiness    146
Wind Speed    146
Country       146
Date          146
dtype: int64

In [15]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
7,Tiarei,PF,84.20,-17.53,-149.33,
8,Avarua,CK,75.20,-21.21,-159.78,
9,Hervey Bay,AU,86.00,-25.30,152.85,
11,Georgetown,GY,75.20,6.80,-58.16,
12,Kapaa,US,78.80,22.08,-159.32,
16,Vaini,IN,75.20,15.34,74.49,
32,Itupiranga,BR,78.80,-5.13,-49.33,
34,Bongandanga,CD,75.29,1.51,21.07,
41,Naze,NG,75.45,5.43,7.07,
42,Atuona,PF,80.58,-9.80,-139.03,


In [19]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}
r=requests.get('base URL', params=parameters)

NameError: name 'parameters' is not defined